### 分离前景和背景

拿到前景的mask

In [1]:
import cv2
import numpy as np

img_dir = "../src/assets/node"

# 加载图像
img = cv2.imread(f'{img_dir}/img.jpeg')

# 如果图像为空，退出
if img is None:
    print("无法加载图像")
    exit()

height, width = img.shape[:2]

gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 设置阈值，进行二值化，保留黑色部分
_, binary_image = cv2.threshold(gray_image, 150, 255, cv2.THRESH_BINARY_INV)

treeMask = binary_image == 255
# hole = binary_image == 255

# removed = img.copy()
tree = img.copy()

# 将原始图像中的白色部分去除
# removed[mask] = (255, 255, 255)  # 将保留部分改为白色
# tree[treeMask] = (255, 255, 255)  # 将保留部分改为白色


transparent_background = np.zeros((height, width, 4), dtype=np.uint8)
transparent_background[:, :, :3] = tree  # 前景 (BGR 通道)
transparent_background[:, :, 3] = binary_image     # Alpha 通道 (255 表示不透明，0 表示透明)
# 保存结果为 PNG 格式，其中透明通道保留
# cv2.imwrite('binary.png', binary_image)
cv2.imwrite(f'{img_dir}/mask.png', transparent_background)
# cv2.imwrite('tree.png', tree)


True

#### ------下方代码不再需要------

根据data里的position来计算每个node的bounding box，之后将二值化的图像根据bbox切割，得到每个node的mask

In [ ]:
import os, json


with open(f"{img_dir}/node.json", "r", encoding="utf-8") as f:
    data = json.load(f)

def traverse_and_store(data, result, path=""):
    if not data:
        return

    # 将当前节点的 name 和 position 存储到结果列表
    result.append({"name": data["name"], "position": data.get("position")})

    # 遍历子节点
    for child in data.get("children", []):
        traverse_and_store(child, result, path)

results = []
traverse_and_store(data, results)

results

In [5]:
import os
import cv2
import json

image_file = f"{img_dir}/mask.png"  # 指定图片路径
output_dir = f"{img_dir}/masks"

image = cv2.imread(image_file, cv2.IMREAD_UNCHANGED)


for item in results:
    name = item["name"]
    bbox = item['position']
    x, y = int(bbox[0][0]), int(bbox[0][1])
    w, h = int(bbox[1][0]), int(bbox[1][1])
    x2, y2 = x + w, y + h

    # 裁切图片
    cropped = image[y:y2, x:x2]

    # 构建输出文件名
    output_path = os.path.join(output_dir, f"{name}.png")

    # 保存裁切的图片
    cv2.imwrite(output_path, cropped)


[ WARN:0@42.964] global loadsave.cpp:268 findDecoder imread_('../src/assets/tree/mask.png'): can't open/read file: check file path/integrity


TypeError: 'NoneType' object is not subscriptable